This Lab is about preprocessing raw Point Clouds with the use of Open3d Python library. Open3d implements several useful functions that facilitate Point Cloud processing and analysis, including downsampling, outlier removal and the estimation of normal vectors at each point. Let's jump in!

First, we import the necessary libraries

In [ ]:
import numpy as np
import pandas as pd
import open3d as o3d
import os
import gc
from pathlib import Path

Specify the working directory and the path to a raw Point Cloud file

In [ ]:
maindir = Path.cwd() / "tutorial_data"
filename = "Sphere_16_75_2_0.6.txt"
path2file = maindir / filename

Let's read the file into a pandas Dataframe first, to have a look at the data. In our case, raw Point Cloud files comprise six columns: three spatial coordinates X, Y, Z for each point, plus the three components I, J, K of the laser orientation vector. 

In [ ]:
colnames = ['X', 'Y', 'Z', 'I', 'J', 'K']
ddf = pd.read_csv(path2file, names=colnames, index_col=False, sep=' ')
ddf

In [ ]:
del ddf
gc.collect()

Now let's load the Point Cloud and render it using the open3d library. 

In [ ]:
pcd = o3d.io.read_point_cloud(str(path2file), format='xyz')
print(pcd)
print(np.asarray(pcd.points))

A better option is to load the Point Cloud using format="xyzn". "n" stands for the normal vector at each point and expects three extra columns, for each of the vector components. A total of six columns is therefore expected when format is set to "xyzn". 

In [ ]:
pcd = o3d.io.read_point_cloud(str(path2file), format='xyzn')
print(pcd)
print(np.asarray(pcd.points))

In [ ]:
print(np.asarray(pcd.normals))

In our case, these "normals" correspond to the laser orientation. Values are identical across the Point Cloud because the sphere was captured with a single orientation. 

In [ ]:
laser_ori = np.asarray(pcd.normals)
print(laser_ori)

Visualize the Point Cloud.

In [ ]:
o3d.visualization.draw_geometries([pcd],
                                 window_name='Sphere 16mm Raw Point Cloud',
                                 width=1920,
                                 height=1080)

Radius outlier removal: Removes points that have neighbors less than nb_points in a sphere of a given radius.

In [ ]:
print("Radius outlier removal")
cloud,ind=pcd.remove_radius_outlier(nb_points=30,radius=0.8)
inlier_cloud=cloud.select_by_index(ind)
outlier_cloud=cloud.select_by_index(ind,invert=True)
print("Showing outliers(red) and inliers(gray):")
outlier_cloud.paint_uniform_color([1,0,0])
inlier_cloud.paint_uniform_color([0.8,0.8,0.8])
o3d.visualization.draw_geometries([inlier_cloud,outlier_cloud],
                                  window_name='Radius outlier removal, Showing outliers(red) and inliers(gray)',
                                  width=1920,
                                  height=1080)

Statistical outlier removal: Removes points that are further away from their neighbors in average. 

In [ ]:
print("Statistical outlier removal")
cloud,ind=pcd.remove_statistical_outlier(nb_neighbors=60,std_ratio=1.0)
inlier_cloud=pcd.select_by_index(ind)
outlier_cloud=pcd.select_by_index(ind,invert=True)
print("Showing outliers(red) and inliers(gray):")
outlier_cloud.paint_uniform_color([1,0,0])
inlier_cloud.paint_uniform_color([0.8,0.8,0.8])
o3d.visualization.draw_geometries([inlier_cloud,outlier_cloud],
                                  window_name='Statistical outlier removal, Showing outliers(red) and inliers(gray)',
                                  width=1920,
                                  height=1080)

In [ ]:
print(pcd)
print(inlier_cloud)

Downsample the Point Cloud via voxelization. Try using different voxel sizes and compare the sizes of the resulting clouds to the raw one. 

In [ ]:
voxel_size = 0.05
print("Downsample the point cloud with a voxel of {}mm".format(voxel_size))
downpcd=inlier_cloud.voxel_down_sample(voxel_size=voxel_size)
o3d.visualization.draw_geometries([downpcd],
                                 window_name='Downsampled sphere, voxel={}mm'.format(voxel_size),
                                 width=1920,
                                 height=1080)

In [ ]:
print(inlier_cloud)
print(downpcd)

Random downsampling, where the sampling ratio: number of selected points to total number of points, a number in the [0-1] interval. 

In [ ]:
print("Random downsampling")
random_down_pcd=inlier_cloud.random_down_sample(sampling_ratio = 0.2)
random_down_pcd.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([random_down_pcd],
                                 window_name='Randomly downsampled Sphere with sampling ratio = 0.2',
                                 width=1920,
                                 height=1080)

In [ ]:
print(inlier_cloud)
print(random_down_pcd)

We keep the inlier cloud and proceed with uniform downsampling, every k points. This way the linear distribution of points in the raw point cloud is preserved.

In [ ]:
k=5
print("Every {} points are selected".format(k))
uni_down_pcd=inlier_cloud.uniform_down_sample(every_k_points=5)
uni_down_pcd.paint_uniform_color([0.5, 0, 0.5])
o3d.visualization.draw_geometries([uni_down_pcd],
                                 window_name='Uniformly downsampled Sphere',
                                 width=1920,
                                 height=1080)

Compare raw to filtered & downsampled Point Cloud size. 

In [ ]:
print(pcd)
print(inlier_cloud)
print(uni_down_pcd)

Recompute the normals of the downsampled Point Cloud. Ensure normal vectors are consistently aligned. 

In [ ]:
uni_down_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.8,max_nn=100 ))

uni_down_pcd.orient_normals_consistent_tangent_plane(100)

frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
o3d.visualization.draw_geometries([uni_down_pcd, frame],
                                 window_name='Recompute the normals of the filtered & downsampled point cloud',
                                 width=1920,
                                 height=1080,
                                 point_show_normal=True)

Retrieve number of points in the final Point Cloud & create initial data array for processed Point Cloud. 

In [ ]:
point_num = len(uni_down_pcd.points)
points_normals_laser = np.concatenate((
                        np.asarray(uni_down_pcd.points), np.asarray(uni_down_pcd.normals), laser_ori[:point_num]), axis=1)
print(points_normals_laser.shape)

Create an initial Dataframe for the processed Point Cloud. This will be the basis for feature extraction. 

In [ ]:
colnames = ['X', 'Y', 'Z', 'Nx', 'Ny', 'Nz', 'I', 'J', 'K']
df = pd.DataFrame(points_normals_laser, columns=colnames)
df

Use laser orientation & the computed normal vectors for feature extraction: Create a function to calculate the Incidence angle of light on the surface at each point. 

In [ ]:
def cosine_incidence(i1, j1, k1, i2, j2, k2):
    n1 = np.sqrt(i1**2 + j1**2 + k1**2)
    n2 = np.sqrt(i2**2 + j2**2 + k2**2)
    dot = np.dot([i1, j1, k1], [i2, j2, k2])
    cos = dot / (n1*n2)
    return np.arccos(cos)

nvec_cos = np.vectorize(cosine_incidence)

Apply the function and save results in a new column of the Dataframe. 

In [ ]:
df['IncAngle'] = nvec_cos(df.Nx, df.Ny, df.Nz, df.I, df.J, df.K)
df

Retrieve geometry, diameter & scanning parameters from Point Cloud filename. 

In [ ]:
sparams = filename[:-4].split('_')
geometry = sparams[0]
diameter = float(sparams[1])
lateral_density = int(sparams[2])
direction_density = int(sparams[3])
exposure_time = float(sparams[4])
print('Geometry: ', geometry)
print('Diameter: ', diameter, ' mm')
print('Lateral Density: ', lateral_density)
print('Direction Density: ', direction_density)
print('Exposure Time: ', exposure_time)

Insert scanning parameters & nominal radius in data array. 

In [ ]:
df['LateralDensity'] = lateral_density
df['DirectionDensity'] = direction_density
df['ExposureTime'] = exposure_time
df['Rnom'] = diameter/2
df

Calculate radial point deviations from nominal surface: this is the target variable. 

In [ ]:
def calculate_radial_point_dev(x, y, z, radius):
    r = np.sqrt(x**2 + y**2 + z**2)
    return float(r - radius)

nvec_dev = np.vectorize(calculate_radial_point_dev)

df['PointDev'] = nvec_dev(df.X, df.Y, df.Z, df.Rnom)
df

Save processed Point Cloud in .csv tabular format, making sure to save the header as well: data directly injestible by ML models!

In [ ]:
savefile = '_'.join(['Processed', filename[:-4]])
savefile += ".csv"
savepath = maindir / savefile
df.to_csv(savepath, sep=',', header=True, index=False)

In [ ]:
del df
gc.collect()